In [4]:
import os
import json
from datetime import date


from sklearn import linear_model
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import scipy



import myfunction
import GDTypeModel as gdt
import MeanValueModel as mvm


with open("./NTU_station.txt", 'r') as F:
    NTU_station = F.readline().split(",")
many_days = os.listdir("./release")[:-1]
train_days = many_days[:-21]
test_days = many_days[-21:]

### GD 訓練時間

29m 17.8s

單純預測 12/18 ~ 12/24 loss 大約 0.48 左右

但是如果同時預測 10/21 ~ 10/24，loss 會飆到 0.86 左右

In [4]:
# NTU_GD_model = gdt.generateGDModels(NTU_station,train_days)
# myfunction.saveModel(NTU_GD_model,"./NTU_GD_model")

In [5]:
sta = "500101181"
model = gdt.GDTypeModel()
cal = myfunction.myDataloader(sta,train_days)
model.training(cal,train_days)

In [3]:
model.GD_model


[  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
   success: True
    status: 0
       fun: 0.4119296312821119
         x: [ 7.456e-02  2.653e-03 ...  2.664e-03  3.840e-01]
       nit: 215
       jac: [ 1.001e-04  1.448e-04 ...  9.737e-05  1.027e-04]
      nfev: 6069
      njev: 289
  hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>,
   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
   success: True
    status: 0
       fun: 0.28605864157753036
         x: [ 7.958e-03  6.331e-02 ...  5.266e-02  6.765e-01]
       nit: 255
       jac: [ 1.512e-04  1.467e-04 ...  9.861e-05  6.768e-05]
      nfev: 7308
      njev: 348
  hess_inv: <20x20 LbfgsInvHessProduct with dtype=float64>,
   message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
   success: True
    status: 0
       fun: 0.3296946286848208
         x: [ 1.420e-01  4.589e-02 ...  4.018e-02  3.803e-01]
       nit: 301
       jac: [-2.064e-04 -1.935e-04 ... -2.307e-04 -2.807e-04]
      nfev: 8337
      njev

In [2]:
NTU_GD_model = myfunction.loadModel("./NTU_GD_model")

In [3]:
interval_1204_1210 = many_days[-21:-14]
interval_1211_1217 = many_days[-14:-7]
interval_1218_1224 = many_days[-7:]

In [5]:
a = gdt.optimizePrediction(NTU_GD_model, NTU_station, interval_1204_1210, interval_1211_1217)
np.mean(a)

0.3526168611802814

In [8]:
np.std(a)

0.13655718290086943

In [6]:
b = gdt.optimizePrediction(NTU_GD_model, NTU_station, interval_1211_1217, interval_1218_1224)
np.mean(b)

0.5107967950123723

In [7]:
np.std(b)

0.28022080191767296

In [3]:
# result_MV = pd.read_csv("./sample_submission_stage3.csv")
result_GD = pd.read_csv("./sample_submission_stage3.csv")

In [4]:
prev_Timestamp = myfunction.getPredictTimeStamp(result_GD,many_days)
pred_GD = gdt.optimizePrediction(NTU_GD_model,NTU_station,prev_Timestamp)
result_GD["sbi"] = pred_GD

In [37]:
prev_Timestamp

['20231007',
 '20231008',
 '20231016',
 '20231017',
 '20231211',
 '20231212',
 '20231213',
 '20231214',
 '20231215',
 '20231216',
 '20231217']

In [5]:
result_GD[["id","sbi"]].to_csv("GD2.csv",index=False)

In [9]:
sta = "500101181"
model = NTU_GD_model[sta].GD_model

In [25]:
# prev = ["20231007","20231008","20231016","20231017"]
# next = ["20231021","20231022","20231023","20231024"]

prev = ["20231211","20231212","20231213","20231214","20231215","20231216","20231217"]
next = ["20231218","20231219","20231220","20231221","20231222","20231223","20231224"]
loss = 0
for sta in NTU_station:
    for i in range(4):
        cal_prev = myfunction.myDataloaderCertainDay(sta,prev[i])
        cal_next = myfunction.myDataloaderCertainDay(sta,next[i])
        X,Y,Y_tot= myfunction.dataCombing(cal_prev,cal_next)
        Y_pred = X @ model[cal_prev[0][3]].x
        loss += myfunction.bicycleError(Y_pred,Y,Y_tot)

TypeError: string indices must be integers

In [23]:
loss / 7 / 112

0.48385385853305574

0.2423584831024353

In [63]:
t2 = []
for i in range(21,1441,20):
    t2.append((cal_1210[:,-3])[i-20:i])
t2 = np.stack(t2,axis=0)
t2.shape

(71, 20)

In [65]:
t2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 2, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 1]])

In [64]:
t2 == t1

C:\Users\shawn\AppData\Local\Temp\ipykernel_20832\2144833118.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  t2 == t1


False